# Evaluation Metrics: SCOT

In [1]:
from shapely.ops import cascaded_union
import matplotlib.pyplot as plt
import geopandas as gpd
import multiprocessing
import pandas as pd
import numpy as np
import skimage.io
import tqdm
import glob
import math
import gdal
import time
import sys
import os
import geopandas as gpd
from shapely import wkt

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib
# matplotlib.use('Agg') # non-interactive

import solaris as sol
from solaris.utils.core import _check_gdf_load
from solaris.raster.image import create_multiband_geotiff 

# import from data_postproc_funcs
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from sn7_baseline_postproc_funcs import map_wrapper, multithread_polys, \
        calculate_iou, track_footprint_identifiers, \
        sn7_convert_geojsons_to_csv
import solaris.eval.scot as scot

/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Midgard/home/hfang

In [2]:
ground_truth = pd.read_csv('/local_storage/users/hfang/inference_winner_split/sn7_baseline_preds/csvs/ground_truth.csv')
aois = [z.split('mosaic_')[-1] for z in ground_truth['filename'].values]
ground_truth['aoi'] = aois
timesteps = [z.split('monthly_')[-1].split('_mosaic')[0] for z in ground_truth['filename'].values]
ground_truth['timestep'] = timesteps
ground_truth = ground_truth.drop('filename', 1)

In [3]:
ground_truth.head()

,id,geometry,aoi,timestep
0,1131,"POLYGON ((1022.739556749351 1023.915693833842,...",L15-0387E-1276N_1549_3087_13,2018_01
1,896,"POLYGON ((839.4174400521442 1023.860588716227,...",L15-0387E-1276N_1549_3087_13,2018_01
2,607,"POLYGON ((977.5393254966475 1021.220368818729,...",L15-0387E-1276N_1549_3087_13,2018_01
3,539,"POLYGON ((804.8580670831725 1021.935737615102,...",L15-0387E-1276N_1549_3087_13,2018_01
4,769,"POLYGON ((817.6027463246137 1022.111423592898,...",L15-0387E-1276N_1549_3087_13,2018_01


In [4]:
ground_truth['geometry'] = ground_truth['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(ground_truth, geometry = 'geometry')
print(gdf)

          id                                           geometry  \
0       1131  POLYGON ((1022.740 1023.916, 1021.780 1025.799...   
1        896  POLYGON ((839.417 1023.861, 832.526 1024.057, ...   
2        607  POLYGON ((977.539 1021.220, 976.830 1024.765, ...   
3        539  POLYGON ((804.858 1021.936, 801.562 1024.604, ...   
4        769  POLYGON ((817.603 1022.111, 814.501 1024.105, ...   
...      ...                                                ...   
953426   199  POLYGON ((16.975 27.234, 23.288 27.566, 23.158...   
953427   215  POLYGON ((264.036 7.059, 267.359 7.059, 267.35...   
953428   225  POLYGON ((269.021 3.875, 273.175 3.875, 273.17...   
953429   648  POLYGON ((267.221 3.459, 267.221 1.659, 269.15...   
953430   625  POLYGON ((268.467 0.829, 268.467 -0.833, 271.7...   

                                 aoi timestep  
0       L15-0387E-1276N_1549_3087_13  2018_01  
1       L15-0387E-1276N_1549_3087_13  2018_01  
2       L15-0387E-1276N_1549_3087_13  2018_01  
3  

In [5]:
pred = pd.read_csv('/Midgard/home/hfang/temporal_CD/HRNet_SN7/submission_hrnet_finetune1.csv')
aois = [z.split('mosaic_')[-1] for z in pred['filename'].values]
pred['aoi'] = aois
timesteps = [z.split('monthly_')[-1].split('_mosaic')[0] for z in pred['filename'].values]
pred['timestep'] = timesteps
pred = pred.drop('filename', 1)

In [6]:
pred.head()

,id,geometry,aoi,timestep
0,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_01
1,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_02
2,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_03
3,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_04
4,0,"POLYGON (( 3 37.3,3 37.7,2.7 37.7,2.7 38.3,2.3...",L15-0632E-0892N_2528_4620_13,2018_05


In [7]:
pred['geometry'] = pred['geometry'].apply(wkt.loads)
pdf = gpd.GeoDataFrame(pred, geometry = 'geometry')
print(pdf)

          id                                           geometry  \
0          0  POLYGON ((3.000 37.300, 3.000 37.700, 2.700 37...   
1          0  POLYGON ((3.000 37.300, 3.000 37.700, 2.700 37...   
2          0  POLYGON ((3.000 37.300, 3.000 37.700, 2.700 37...   
3          0  POLYGON ((3.000 37.300, 3.000 37.700, 2.700 37...   
4          0  POLYGON ((3.000 37.300, 3.000 37.700, 2.700 37...   
...      ...                                                ...   
592180  2011  POLYGON ((874.700 1015.000, 874.700 1015.300, ...   
592181  2011  POLYGON ((874.700 1015.000, 874.700 1015.300, ...   
592182  2011  POLYGON ((874.700 1015.000, 874.700 1015.300, ...   
592183  2011  POLYGON ((874.700 1015.000, 874.700 1015.300, ...   
592184  2011  POLYGON ((874.700 1015.000, 874.700 1015.300, ...   

                                 aoi timestep  
0       L15-0632E-0892N_2528_4620_13  2018_01  
1       L15-0632E-0892N_2528_4620_13  2018_02  
2       L15-0632E-0892N_2528_4620_13  2018_03  
3  

In [8]:
scot.scot_multi_aoi(gdf, pdf, verbose=True)


1 / 10: AOI L15-0387E-1276N_1549_3087_13
Matching footprints
   1: F1 = 0.4117
   2: F1 = 0.4425
   3: F1 = 0.4423
   4: F1 = 0.4446
   5: F1 = 0.4415
   6: F1 = 0.4418
   7: F1 = 0.4411
   8: F1 = 0.4420
   9: F1 = 0.4442
  10: F1 = 0.4440
  11: F1 = 0.4357
  12: F1 = 0.4319
  13: F1 = 0.4448
  14: F1 = 0.4469
  15: F1 = 0.4464
  16: F1 = 0.4466
  17: F1 = 0.4463
  18: F1 = 0.4475
  19: F1 = 0.4476
  20: F1 = 0.4393
  21: F1 = 0.4356
  22: F1 = 0.4256
Identifying mismatches
Tracking:
    Mismatches: 3
      True Pos: 12879
     False Pos: 8465
     False Neg: 24206
   Track Score: 0.4408
Change Detection:
      True Pos: 152
     False Pos: 381
     False Neg: 355
  Change Score: 0.2923
Combined Score: 0.4002

2 / 10: AOI L15-0566E-1185N_2265_3451_13
Matching footprints
   1: F1 = 0.2373
   2: F1 = 0.1935
   3: F1 = 0.1935
   4: F1 = 0.2000
   5: F1 = 0.2466
   6: F1 = 0.2716
   7: F1 = 0.2759
   8: F1 = 0.2815
   9: F1 = 0.3056
  10: F1 = 0.4876
  11: F1 = 0.5000
  12: F1 = 0.5091
 

(0.3272983838598892,
 {'L15-0387E-1276N_1549_3087_13': [3,
   12879,
   8465,
   24206,
   0.4408427322048983,
   152,
   381,
   355,
   0.2923076923076923,
   0.4001734132215971],
  'L15-0566E-1185N_2265_3451_13': [7,
   2355,
   1329,
   2834,
   0.5308238476276343,
   129,
   300,
   530,
   0.23713235294117646,
   0.4254410066900928],
  'L15-0632E-0892N_2528_4620_13': [1,
   23791,
   26650,
   40049,
   0.41635967483658703,
   38,
   220,
   95,
   0.19437340153452684,
   0.33894134040375595],
  'L15-1015E-1062N_4061_3941_13': [9,
   31795,
   14751,
   25035,
   0.6151331063302894,
   183,
   775,
   495,
   0.2237163814180929,
   0.4556803884523016],
  'L15-1200E-0847N_4802_4803_13': [2,
   94096,
   53313,
   172218,
   0.4548743966373636,
   34,
   221,
   156,
   0.15280898876404495,
   0.3259929727461795],
  'L15-1276E-1107N_5105_3761_13': [2,
   28814,
   13298,
   44336,
   0.49997397234127466,
   74,
   590,
   643,
   0.10716871832005793,
   0.28849209290376365],
  'L15